# NaonedIA - Expérience Logement

Hector Basset - Ippon Technologies

Analyse des données du cadastre en vue de la sélection d'un dataset.

In [1]:
import pandas as pd
import numpy as np
from os import path

import plotly.offline as py
import plotly.figure_factory as ff
import plotly.graph_objs as go

In [2]:
py.init_notebook_mode(connected=True)

def py_table(data, filename):
    table = ff.create_table(data)
    py.iplot(table, filename=filename)

def py_pie(values, title, filename, labels=None):
    if labels is None:
        labels = values.index
        values = values.values
    pie = go.Pie(labels=labels, values=values, title=title)
    py.iplot([pie], filename=filename)

In [3]:
def get_code_postaux():
    return pd.read_csv(
        'data/code-postaux.csv',
        delimiter=';',
        header=0,
        usecols=[
            'Nom_commune'
        ],
        index_col=False,
        squeeze=True
    ).unique()

def get_data_year(year, cp):
    data_year = pd.read_csv(
        'data/valeursfoncieres-%i.txt.xz' % year,
        delimiter='|',
        header=0,
        usecols=[
            'Date mutation',
            'Nature mutation',
            'Valeur fonciere',
            'No voie',
            'B/T/Q',
            'Type de voie',
            'Voie',
            'Code postal',
            'Commune',
            'Surface Carrez du 1er lot',
            'Surface Carrez du 2eme lot',
            'Surface Carrez du 3eme lot',
            'Surface Carrez du 4eme lot',
            'Surface Carrez du 5eme lot',
            'Nombre de lots',
            'Type local',
            'Surface reelle bati',
            'Nombre pieces principales',
            'Surface terrain'
        ],
        index_col=False,
        decimal=',',
        iterator=True
    )

    data_year = pd.concat([chunk[chunk['Code postal'].astype('str').str.startswith('44') & chunk['Commune'].isin(cp)] for chunk in data_year], ignore_index=True)

    return data_year

def get_data():
    cp = get_code_postaux()

    data = pd.concat([get_data_year(year, cp) for year in [2014, 2015, 2016, 2017, 2018]], ignore_index=True)

    data['Nature mutation'] = data['Nature mutation'].astype('category')
    data['Valeur fonciere'] = data['Valeur fonciere'].astype('float64')
    data['No voie'] = data['No voie'].astype('str')
    data['No voie'] = data['No voie'].apply(lambda n: n.split('.')[0])
    data['B/T/Q'] = data['B/T/Q'].astype('category')
    data['Type de voie'] = data['Type de voie'].astype('category')
    data['Voie'] = data['Voie'].astype('str')
    data['Code postal'] = data['Code postal'].astype('str')
    data['Code postal'] = data['Code postal'].apply(lambda n: n.split('.')[0])
    data['Code postal'] = data['Code postal'].astype('category')
    data['Commune'] = data['Commune'].astype('category')
    data['Surface Carrez du 1er lot'] = data['Surface Carrez du 1er lot'].astype('float64')
    data['Surface Carrez du 2eme lot'] = data['Surface Carrez du 2eme lot'].astype('float64')
    data['Surface Carrez du 3eme lot'] = data['Surface Carrez du 3eme lot'].astype('float64')
    data['Surface Carrez du 4eme lot'] = data['Surface Carrez du 4eme lot'].astype('float64')
    data['Surface Carrez du 5eme lot'] = data['Surface Carrez du 5eme lot'].astype('float64')
    data['Nombre de lots'] = data['Nombre de lots'].astype('int64')
    data['Type local'] = data['Type local'].astype('category')
    data['Surface reelle bati'] = data['Surface reelle bati'].astype('float64')
    data['Nombre pieces principales'] = data['Nombre pieces principales'].astype('float64')
    data['Surface terrain'] = data['Surface terrain'].astype('float64')

    return data

## Chargement des données relatives à Nantes Métropole

In [4]:
df = None
if path.isfile('store/all.pkl.xz'):
    df = pd.read_pickle('store/all.pkl.xz')
else:
    df = get_data()
    df.to_pickle('store/all.pkl.xz')

print("%i observations chargées" % df.shape[0])
df.head(10)

147010 observations chargées


,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Voie,Code postal,Commune,Surface Carrez du 1er lot,Surface Carrez du 2eme lot,Surface Carrez du 3eme lot,Surface Carrez du 4eme lot,Surface Carrez du 5eme lot,Nombre de lots,Type local,Surface reelle bati,Nombre pieces principales,Surface terrain
0,08/01/2014,Vente,29000.00,7,NaN,AV,BASCHER,44000,NANTES,NaN,NaN,NaN,NaN,NaN,0,Dépendance,0.0,0.0,26.0
1,08/01/2014,Vente,29000.00,7,NaN,AV,BASCHER,44000,NANTES,NaN,NaN,NaN,NaN,NaN,0,Dépendance,0.0,0.0,26.0
2,09/01/2014,Vente en l'état futur d'achèvement,285000.00,29,B,RUE,DE LA PELLETERIE,44000,NANTES,NaN,NaN,NaN,NaN,NaN,1,Dépendance,0.0,0.0,NaN
3,09/01/2014,Vente en l'état futur d'achèvement,285000.00,29,B,RUE,DE LA PELLETERIE,44000,NANTES,NaN,NaN,NaN,NaN,NaN,1,Appartement,65.0,3.0,NaN
4,07/01/2014,Vente en l'état futur d'achèvement,329096.99,5,NaN,PL,ARISTIDE BRIAND,44000,NANTES,NaN,NaN,NaN,NaN,NaN,1,Appartement,73.0,3.0,NaN
5,07/01/2014,Vente en l'état futur d'achèvement,329096.99,5,NaN,PL,ARISTIDE BRIAND,44000,NANTES,NaN,NaN,NaN,NaN,NaN,1,Dépendance,0.0,0.0,NaN
6,08/01/2014,Vente,192.00,nan,NaN,NaN,LES CLOSEAUX,44800,ST-HERBLAIN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,24.0
7,02/01/2014,Vente,194400.00,23,NaN,BD,EMILE ROMANET,44100,NANTES,NaN,NaN,NaN,NaN,NaN,1,Dépendance,0.0,0.0,NaN
8,02/01/2014,Vente,194400.00,23,NaN,BD,EMILE ROMANET,44100,NANTES,NaN,NaN,NaN,NaN,NaN,1,Appartement,84.0,4.0,NaN
9,02/01/2014,Vente,107000.00,11,NaN,RUE,DU DOCTEUR ALFRED CORLAY,44800,ST-HERBLAIN,NaN,NaN,NaN,NaN,NaN,1,Dépendance,0.0,0.0,NaN


In [5]:
print("%i variables" % df.shape[1])
pd.DataFrame(df.columns, columns=['Colonnes'])

19 variables


,Colonnes
0,Date mutation
1,Nature mutation
2,Valeur fonciere
3,No voie
4,B/T/Q
5,Type de voie
6,Voie
7,Code postal
8,Commune
9,Surface Carrez du 1er lot


## Analyse des natures de mutation

In [6]:
py_pie(df.groupby(['Nature mutation']).size(), 'Natures de mutation dans le dataset', 'natures')

Ce dataset inclue des mutations autres qu'une simple vente (qui représente cependant la grande majorité des cas). Pour simplifier et se limiter aux cas d'utilisation les plus fréquents, nous n'allons garder que les ventes, et retirer la variable `Nature mutation`.

In [7]:
df.drop(df[df['Nature mutation'] != 'Vente'].index, inplace=True)
df.drop('Nature mutation', axis=1, inplace=True)

In [8]:
print("%i observations restantes" % df.shape[0])
df.head(10)

113237 observations restantes


,Date mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Voie,Code postal,Commune,Surface Carrez du 1er lot,Surface Carrez du 2eme lot,Surface Carrez du 3eme lot,Surface Carrez du 4eme lot,Surface Carrez du 5eme lot,Nombre de lots,Type local,Surface reelle bati,Nombre pieces principales,Surface terrain
0,08/01/2014,29000.0,7,NaN,AV,BASCHER,44000,NANTES,NaN,NaN,NaN,NaN,NaN,0,Dépendance,0.0,0.0,26.0
1,08/01/2014,29000.0,7,NaN,AV,BASCHER,44000,NANTES,NaN,NaN,NaN,NaN,NaN,0,Dépendance,0.0,0.0,26.0
6,08/01/2014,192.0,nan,NaN,NaN,LES CLOSEAUX,44800,ST-HERBLAIN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,24.0
7,02/01/2014,194400.0,23,NaN,BD,EMILE ROMANET,44100,NANTES,NaN,NaN,NaN,NaN,NaN,1,Dépendance,0.0,0.0,NaN
8,02/01/2014,194400.0,23,NaN,BD,EMILE ROMANET,44100,NANTES,NaN,NaN,NaN,NaN,NaN,1,Appartement,84.0,4.0,NaN
9,02/01/2014,107000.0,11,NaN,RUE,DU DOCTEUR ALFRED CORLAY,44800,ST-HERBLAIN,NaN,NaN,NaN,NaN,NaN,1,Dépendance,0.0,0.0,NaN
10,02/01/2014,107000.0,11,NaN,RUE,DU DOCTEUR ALFRED CORLAY,44800,ST-HERBLAIN,45.80,NaN,NaN,NaN,NaN,1,Appartement,46.0,2.0,NaN
11,08/01/2014,295000.0,13,NaN,RUE,DE L ARCHE SECHE,44000,NANTES,NaN,89.2,NaN,NaN,NaN,2,Appartement,105.0,2.0,NaN
12,09/01/2014,208154.0,38,NaN,RUE,DE LA PLANCHE AU GUE,44300,NANTES,103.64,NaN,NaN,NaN,NaN,1,Appartement,103.0,5.0,NaN
13,09/01/2014,208154.0,38,NaN,RUE,DE LA PLANCHE AU GUE,44300,NANTES,NaN,NaN,NaN,NaN,NaN,1,Dépendance,0.0,0.0,NaN


## Analyse des types de locaux

In [9]:
py_pie(df.groupby(['Type local']).size(), 'Types de locaux dans le dataset', 'locaux')

Là aussi, pour simplifier et adresser le plus grand nombre de cas d'utilisation, nous allons nous limiter aux appartements et aux maisons.

In [10]:
df.drop(df[~df['Type local'].isin(['Appartement', 'Maison'])].index, inplace=True)
df['Type local'].cat.remove_unused_categories(inplace=True)

In [11]:
print("%i observations restantes" % df.shape[0])
df.head(10)

57565 observations restantes


,Date mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Voie,Code postal,Commune,Surface Carrez du 1er lot,Surface Carrez du 2eme lot,Surface Carrez du 3eme lot,Surface Carrez du 4eme lot,Surface Carrez du 5eme lot,Nombre de lots,Type local,Surface reelle bati,Nombre pieces principales,Surface terrain
8,02/01/2014,194400.0,23,NaN,BD,EMILE ROMANET,44100,NANTES,NaN,NaN,NaN,NaN,NaN,1,Appartement,84.0,4.0,NaN
10,02/01/2014,107000.0,11,NaN,RUE,DU DOCTEUR ALFRED CORLAY,44800,ST-HERBLAIN,45.80,NaN,NaN,NaN,NaN,1,Appartement,46.0,2.0,NaN
11,08/01/2014,295000.0,13,NaN,RUE,DE L ARCHE SECHE,44000,NANTES,NaN,89.2,NaN,NaN,NaN,2,Appartement,105.0,2.0,NaN
12,09/01/2014,208154.0,38,NaN,RUE,DE LA PLANCHE AU GUE,44300,NANTES,103.64,NaN,NaN,NaN,NaN,1,Appartement,103.0,5.0,NaN
15,06/01/2014,79000.0,8,NaN,RUE,DES CARMELITES,44000,NANTES,26.45,NaN,NaN,NaN,NaN,1,Appartement,25.0,1.0,NaN
18,10/01/2014,40000.0,21,NaN,BD,VICTOR HUGO,44200,NANTES,NaN,NaN,NaN,NaN,NaN,2,Appartement,19.0,1.0,NaN
19,02/01/2014,335000.0,23,NaN,RUE,DES CANARIS,44300,NANTES,NaN,NaN,NaN,NaN,NaN,0,Maison,118.0,5.0,562.0
20,04/01/2014,67000.0,5,NaN,BD,HONORE DE BALZAC,44100,NANTES,22.42,NaN,NaN,NaN,NaN,1,Appartement,22.0,1.0,NaN
22,06/01/2014,230000.0,6,NaN,AV,DES CIGALES,44300,NANTES,NaN,NaN,NaN,NaN,NaN,0,Maison,99.0,4.0,269.0
24,03/01/2014,160000.0,25,NaN,RUE,LOUIS BROCHU,44230,ST SEBASTIEN SUR LOIRE,NaN,NaN,NaN,NaN,NaN,0,Maison,68.0,4.0,434.0


## Analyse du nombre de lots

In [12]:
py_pie(df.groupby(['Nombre de lots']).size(), 'Nombres de lots de chaque bien dans le dataset', 'lots')

De nombreux biens sont vendus en lots, et la valeur foncière représente alors le total des lots. Là aussi dans un but de simplification nous n'allons garder que les biens contenant 0 ou 1 lot, et retirer les variables `Nombre de lots` et `Surface Carrez du 2eme lot` et plus.

In [13]:
df.drop(df[df['Nombre de lots'] > 1].index, inplace=True)
df.drop('Nombre de lots', axis=1, inplace=True)
df.drop('Surface Carrez du 2eme lot', axis=1, inplace=True)
df.drop('Surface Carrez du 3eme lot', axis=1, inplace=True)
df.drop('Surface Carrez du 4eme lot', axis=1, inplace=True)
df.drop('Surface Carrez du 5eme lot', axis=1, inplace=True)

In [14]:
print("%i observations restantes" % df.shape[0])
df.head(10)

42194 observations restantes


,Date mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Voie,Code postal,Commune,Surface Carrez du 1er lot,Type local,Surface reelle bati,Nombre pieces principales,Surface terrain
8,02/01/2014,194400.0,23,NaN,BD,EMILE ROMANET,44100,NANTES,NaN,Appartement,84.0,4.0,NaN
10,02/01/2014,107000.0,11,NaN,RUE,DU DOCTEUR ALFRED CORLAY,44800,ST-HERBLAIN,45.80,Appartement,46.0,2.0,NaN
12,09/01/2014,208154.0,38,NaN,RUE,DE LA PLANCHE AU GUE,44300,NANTES,103.64,Appartement,103.0,5.0,NaN
15,06/01/2014,79000.0,8,NaN,RUE,DES CARMELITES,44000,NANTES,26.45,Appartement,25.0,1.0,NaN
19,02/01/2014,335000.0,23,NaN,RUE,DES CANARIS,44300,NANTES,NaN,Maison,118.0,5.0,562.0
20,04/01/2014,67000.0,5,NaN,BD,HONORE DE BALZAC,44100,NANTES,22.42,Appartement,22.0,1.0,NaN
22,06/01/2014,230000.0,6,NaN,AV,DES CIGALES,44300,NANTES,NaN,Maison,99.0,4.0,269.0
24,03/01/2014,160000.0,25,NaN,RUE,LOUIS BROCHU,44230,ST SEBASTIEN SUR LOIRE,NaN,Maison,68.0,4.0,434.0
28,10/01/2014,172000.0,4,NaN,BD,BOULAY PATY,44100,NANTES,75.64,Appartement,74.0,3.0,NaN
33,10/01/2014,250000.0,2,NaN,RUE,BOSSUET,44000,NANTES,NaN,Appartement,19.0,1.0,NaN


## Suppression des doublons

In [15]:
df.drop_duplicates(inplace=True)

In [16]:
print("%i observations restantes" % df.shape[0])
df.head(10)

40281 observations restantes


,Date mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Voie,Code postal,Commune,Surface Carrez du 1er lot,Type local,Surface reelle bati,Nombre pieces principales,Surface terrain
8,02/01/2014,194400.0,23,NaN,BD,EMILE ROMANET,44100,NANTES,NaN,Appartement,84.0,4.0,NaN
10,02/01/2014,107000.0,11,NaN,RUE,DU DOCTEUR ALFRED CORLAY,44800,ST-HERBLAIN,45.80,Appartement,46.0,2.0,NaN
12,09/01/2014,208154.0,38,NaN,RUE,DE LA PLANCHE AU GUE,44300,NANTES,103.64,Appartement,103.0,5.0,NaN
15,06/01/2014,79000.0,8,NaN,RUE,DES CARMELITES,44000,NANTES,26.45,Appartement,25.0,1.0,NaN
19,02/01/2014,335000.0,23,NaN,RUE,DES CANARIS,44300,NANTES,NaN,Maison,118.0,5.0,562.0
20,04/01/2014,67000.0,5,NaN,BD,HONORE DE BALZAC,44100,NANTES,22.42,Appartement,22.0,1.0,NaN
22,06/01/2014,230000.0,6,NaN,AV,DES CIGALES,44300,NANTES,NaN,Maison,99.0,4.0,269.0
24,03/01/2014,160000.0,25,NaN,RUE,LOUIS BROCHU,44230,ST SEBASTIEN SUR LOIRE,NaN,Maison,68.0,4.0,434.0
28,10/01/2014,172000.0,4,NaN,BD,BOULAY PATY,44100,NANTES,75.64,Appartement,74.0,3.0,NaN
33,10/01/2014,250000.0,2,NaN,RUE,BOSSUET,44000,NANTES,NaN,Appartement,19.0,1.0,NaN


## Données manquantes

Observons le taux de remplissage des colonnes :

In [17]:
pd.DataFrame((df.count() / df.shape[0] * 100).apply(lambda p: "%.2f %%" % p), columns=['Taux de remplissage'])

,Taux de remplissage
Date mutation,100.00 %
Valeur fonciere,99.74 %
No voie,100.00 %
B/T/Q,6.19 %
Type de voie,97.31 %
Voie,100.00 %
Code postal,100.00 %
Commune,100.00 %
Surface Carrez du 1er lot,29.75 %
Type local,100.00 %


La valeur foncière n'est pas toujours remplie, or il s'agit de la variable que l'on cherche à prédire. On supprime donc les observations concernées.

In [18]:
df.drop(df[df['Valeur fonciere'].isna()].index, inplace=True)

In [19]:
print("%i observations restantes" % df.shape[0])
df.head(10)

40178 observations restantes


,Date mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Voie,Code postal,Commune,Surface Carrez du 1er lot,Type local,Surface reelle bati,Nombre pieces principales,Surface terrain
8,02/01/2014,194400.0,23,NaN,BD,EMILE ROMANET,44100,NANTES,NaN,Appartement,84.0,4.0,NaN
10,02/01/2014,107000.0,11,NaN,RUE,DU DOCTEUR ALFRED CORLAY,44800,ST-HERBLAIN,45.80,Appartement,46.0,2.0,NaN
12,09/01/2014,208154.0,38,NaN,RUE,DE LA PLANCHE AU GUE,44300,NANTES,103.64,Appartement,103.0,5.0,NaN
15,06/01/2014,79000.0,8,NaN,RUE,DES CARMELITES,44000,NANTES,26.45,Appartement,25.0,1.0,NaN
19,02/01/2014,335000.0,23,NaN,RUE,DES CANARIS,44300,NANTES,NaN,Maison,118.0,5.0,562.0
20,04/01/2014,67000.0,5,NaN,BD,HONORE DE BALZAC,44100,NANTES,22.42,Appartement,22.0,1.0,NaN
22,06/01/2014,230000.0,6,NaN,AV,DES CIGALES,44300,NANTES,NaN,Maison,99.0,4.0,269.0
24,03/01/2014,160000.0,25,NaN,RUE,LOUIS BROCHU,44230,ST SEBASTIEN SUR LOIRE,NaN,Maison,68.0,4.0,434.0
28,10/01/2014,172000.0,4,NaN,BD,BOULAY PATY,44100,NANTES,75.64,Appartement,74.0,3.0,NaN
33,10/01/2014,250000.0,2,NaN,RUE,BOSSUET,44000,NANTES,NaN,Appartement,19.0,1.0,NaN


In [20]:
pd.DataFrame((df.count() / df.shape[0] * 100).apply(lambda p: "%.2f %%" % p), columns=['Taux de remplissage'])

,Taux de remplissage
Date mutation,100.00 %
Valeur fonciere,100.00 %
No voie,100.00 %
B/T/Q,6.19 %
Type de voie,97.31 %
Voie,100.00 %
Code postal,100.00 %
Commune,100.00 %
Surface Carrez du 1er lot,29.83 %
Type local,100.00 %


La surface Carrez n'est renseignée que dans moins de 30 % des cas, et la surface du terrain dans moins de 60 %.

Proposition :
* supprimer la surface Carrez, et n'utiliser que la surface batie ;
* utiliser la surface batie à la place de celle du terrain là où cette dernière n'est pas remplie.

Toutes les autres variables sont remplies à 100 %, hormis `B/T/Q` et `Type de voie` mais ça n'est pas grave car une adresse ne contient pas forcement un numéro ou un type de voie (cas des maisons isolées par exemple).

In [21]:
# Nombre pieces principales est maintenant rempli à 100 %, on peut le convertir en int
df['Nombre pieces principales'] = df['Nombre pieces principales'].astype('int64')

df.head(10)

,Date mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Voie,Code postal,Commune,Surface Carrez du 1er lot,Type local,Surface reelle bati,Nombre pieces principales,Surface terrain
8,02/01/2014,194400.0,23,NaN,BD,EMILE ROMANET,44100,NANTES,NaN,Appartement,84.0,4,NaN
10,02/01/2014,107000.0,11,NaN,RUE,DU DOCTEUR ALFRED CORLAY,44800,ST-HERBLAIN,45.80,Appartement,46.0,2,NaN
12,09/01/2014,208154.0,38,NaN,RUE,DE LA PLANCHE AU GUE,44300,NANTES,103.64,Appartement,103.0,5,NaN
15,06/01/2014,79000.0,8,NaN,RUE,DES CARMELITES,44000,NANTES,26.45,Appartement,25.0,1,NaN
19,02/01/2014,335000.0,23,NaN,RUE,DES CANARIS,44300,NANTES,NaN,Maison,118.0,5,562.0
20,04/01/2014,67000.0,5,NaN,BD,HONORE DE BALZAC,44100,NANTES,22.42,Appartement,22.0,1,NaN
22,06/01/2014,230000.0,6,NaN,AV,DES CIGALES,44300,NANTES,NaN,Maison,99.0,4,269.0
24,03/01/2014,160000.0,25,NaN,RUE,LOUIS BROCHU,44230,ST SEBASTIEN SUR LOIRE,NaN,Maison,68.0,4,434.0
28,10/01/2014,172000.0,4,NaN,BD,BOULAY PATY,44100,NANTES,75.64,Appartement,74.0,3,NaN
33,10/01/2014,250000.0,2,NaN,RUE,BOSSUET,44000,NANTES,NaN,Appartement,19.0,1,NaN


In [22]:
df.to_pickle('store/after_analyze.pkl.xz')

## Conclusion de l'analyse

En utilisant ce dataset, un fois nettoyé, nous disposons d'une base solide de plus de 40 000 observations contenant des données réelles (non estimées par le vendeur par exemple). De plus, le taux de remplissage des variables est élevé.

Le seul bémol est le nombre un peu trop petit de variables. Cependant, il est tout à fait possible d'utiliser les informations de ce dataset pour l'enrichir et ajouter des variables qui pourraient être pertinentes. Je fais 2 propositions dans ce sens ci-dessous.

## Exploitation des adresses

Les adresses sont renseignées avec précision. Nous pourrions donc les utiliser afin d'enrichir le dataset de variables telles que : Distance de l'arrêt de tram le plus proche, de l'arrêt de bus, de la gare, d'une pharmacie, d'un supermarché, d'un groupe de commerces, etc. Le choix des variables est à arrêter en fonction des informations que l'on souhaite demander à l'utilisateur et de ce qu'il sera possible de faire pour enrichir le dataset via OpenStreetMap par exemple. Il peut également être intéressant de renseigner les distances à pied ou en voiture.

Une fois le dataset enrichi, nous supprimerons les adresses des données car le but est de ne pas stocker des informations trop précises (pour ne pas inquiéter l'utilisateur).

Pour éviter à l'utilisateur d'avoir à saisir toutes ces informations, on pourrais lui demander de saisir directement l'adresse du bien, en lui précisant bien qu'elle ne sera ni stockée ni utilisée pour la prédiction mais uniquement pour déduire les informations citées plus haut (et avec plus de précision que si c'est lui qui le fait du coup).

Nantes Métropole fournit de nombreux datasets en open data que l'on pourrai utiliser :
* [localisation des arrêts de tram et de bus](https://data.nantesmetropole.fr/explore/dataset/244400404_tan-arrets/table/) ;
* [localisation des entreprises](https://data.nantesmetropole.fr/explore/dataset/244400404_base-sirene-entreprises-nantes-metropole/table/) ;
* ...

## Exploitation de la date

Les dates de vente sont elles aussi renseignées avec précision. Nous pourrions les utiliser afin d'enrichir le dataset de variables telles que : prix moyen du mètre carré à cette date, tension du marché à cette date, revenu moyen par habitant à cette date, densité de population, etc. À renseigner pour l'année courante ou le mois courant (ou précédent), au niveau national ou local en fonction de ce que l'on peut trouver comme historique sur le net.

L'utilisateur n'aura bien entendu pas à rentrer ces données lorsqu'il voudra estimer un bien, mais on viendrait automatiquement enrichir sa saisie avec les derniers chiffres connus.